<a href="https://colab.research.google.com/github/hirokimituya/book_create-profitable-AI-with-Python/blob/main/ch05_04_item_recommend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 5.4 お勧め商品の提案(アソシエーション分析)

## 共通事前処理


In [1]:
# 日本語化ライブラリ導入
!pip install japanize-matplotlib | tail -n 1

In [2]:
# 共通事前処理

# 余分なワーニングを非表示にする
import warnings
warnings.filterwarnings('ignore')

# 必要ライブラリのimport
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# matplotlib日本語化対応
import japanize_matplotlib

# データフレーム表示用関数
from IPython.display import display

# 表示オプション調整
# numpyの浮動小数点の表示精度
np.set_printoptions(suppress=True, precision=4)

# pandasでの浮動小数点の表示精度
pd.options.display.float_format = '{:.4f}'.format

# データフレームですべての項目を表示
pd.set_option("display.max_columns",None)

# グラフのデフォルトフォント指定
plt.rcParams["font.size"] = 14

# 乱数の種
random_seed = 123

## 5.4.4 データ読み込みからデータ確認まで

In [3]:
# データ読み込み
# 公開データはExcel形式なので、read_excel関数で直接読み込む
# 時間がかなりかかるので注意してください (ファイルサイズ 約23MB)
df = pd.read_excel('http://archive.ics.uci.edu/ml/\
machine-learning-databases/00352/Online%20Retail.xlsx')

# 項目名を日本語に変更
columns = [
    '発注番号', '商品番号', '商品説明', '商品個数', '明細書発行日', 
    '商品単価', '顧客番号', '国名'
]
df.columns = columns

In [4]:
# データ件数確認
print(df.shape[0])

# データ内容確認
display(df.head())

541909


発注番号    商品番号                                 商品説明  商品個数  \
0  536365  85123A   WHITE HANGING HEART T-LIGHT HOLDER     6   
1  536365   71053                  WHITE METAL LANTERN     6   
2  536365  84406B       CREAM CUPID HEARTS COAT HANGER     8   
3  536365  84029G  KNITTED UNION FLAG HOT WATER BOTTLE     6   
4  536365  84029E       RED WOOLLY HOTTIE WHITE HEART.     6   

               明細書発行日   商品単価       顧客番号              国名  
0 2010-12-01 08:26:00 2.5500 17850.0000  United Kingdom  
1 2010-12-01 08:26:00 3.3900 17850.0000  United Kingdom  
2 2010-12-01 08:26:00 2.7500 17850.0000  United Kingdom  
3 2010-12-01 08:26:00 3.3900 17850.0000  United Kingdom  
4 2010-12-01 08:26:00 3.3900 17850.0000  United Kingdom

In [5]:
# 欠損値確認
print(df.isnull().sum())

発注番号           0
商品番号           0
商品説明        1454
商品個数           0
明細書発行日         0
商品単価           0
顧客番号      135080
国名             0
dtype: int64


In [6]:
# 国名確認
print(df['国名'].value_counts().head(10))

United Kingdom    495478
Germany             9495
France              8557
EIRE                8196
Spain               2533
Netherlands         2371
Belgium             2069
Switzerland         2002
Portugal            1519
Australia           1259
Name: 国名, dtype: int64


## 5.4.5 データ前処理

In [9]:
# 「発注種別」列の追加

# 前処理用にデータをコピー
df2 = df.copy()

# 発注番号の頭1桁を別項目に抽出
# (5: 新規オーダー, C: キャンセル)
df2['発注種別'] = df['発注番号'].map(lambda x: str(x)[0])

# 結果確認
display(df2.head())

# 種別個数確認
print(df2['発注種別'].value_counts())

発注番号    商品番号                                 商品説明  商品個数  \
0  536365  85123A   WHITE HANGING HEART T-LIGHT HOLDER     6   
1  536365   71053                  WHITE METAL LANTERN     6   
2  536365  84406B       CREAM CUPID HEARTS COAT HANGER     8   
3  536365  84029G  KNITTED UNION FLAG HOT WATER BOTTLE     6   
4  536365  84029E       RED WOOLLY HOTTIE WHITE HEART.     6   

               明細書発行日   商品単価       顧客番号              国名 発注種別  
0 2010-12-01 08:26:00 2.5500 17850.0000  United Kingdom    5  
1 2010-12-01 08:26:00 3.3900 17850.0000  United Kingdom    5  
2 2010-12-01 08:26:00 2.7500 17850.0000  United Kingdom    5  
3 2010-12-01 08:26:00 3.3900 17850.0000  United Kingdom    5  
4 2010-12-01 08:26:00 3.3900 17850.0000  United Kingdom    5

5    532618
C      9288
A         3
Name: 発注種別, dtype: int64


In [10]:
# 新規オーダーのみ抽出
df2 = df2[df2['発注種別'] == '5']

# 件数確認
print(df2.shape[0])

532618


In [11]:
# 分析対象をフランスに限定する
df3 = df2[df2['国名'] == 'France']

# 件数確認
print(df3.shape[0])

8408


In [12]:
## 発注番号と商品番号をキーに商品個数を集計する
w1 = df3.groupby(['発注番号', '商品番号'])['商品個数'].sum()

# 結果確認
display(w1.head())

発注番号    商品番号 
536370  10002    48
        21035    18
        21724    12
        21731    24
        21791    24
Name: 商品個数, dtype: int64

In [13]:
# 商品番号を列に移動（unstack関数の利用）
w2 = w1.unstack().reset_index().fillna(0).set_index('発注番号')

# サイズ確認
print(w2.shape)

# 結果確認
display(w2.head())

(392, 1542)


商品番号     10002  10120  10125  10135  11001  15036  15039  16012  16048  16218  \
発注番号                                                                            
536370 48.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000   
536852  0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000   
536974  0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000   
537065  0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000   
537463  0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000   

商品番号    16219  16225  16236  16237   16238  17174  20615  20617  20658  20665  \
発注番号                                                                            
536370 0.0000 0.0000 0.0000 0.0000  0.0000 0.0000 0.0000 0.0000 0.0000 0.0000   
536852 0.0000 0.0000 0.0000 0.0000  0.0000 0.0000 0.0000 0.0000 0.0000 0.0000   
536974 0.0000 0.0000 0.0000 0.0000  0.0000 0.0000 0.0000 0.0000 0.0000 0.0000   
537065 0.0000 0.0000 0.0000 0.0000 28.0000 0.0000 0.0000 0.0000 0.0000 6.0000   
537463 0.0000 0.0000 0.0000 0.0000  0.0000 0.0000 0.0000 0.0000 0.0000 0.0000   

商品番号    20668  20674  20675  20676  20677  20679  20681  20682  20684  20685  \
発注番号                                                                           
536370 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000   
536852 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000   
536974 0.0000 0.0000 0.0000 0.0000 0.0000 6.0000 0.0000 0.0000 0.0000 0.0000   
537065 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000   
537463 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 6.0000 6.0000 0.0000 0.0000   

商品番号    20686  20702  20704  20711  20712  20713  20717  20718   20719  20723  \
発注番号                                                                            
536370 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000  0.0000 0.0000   
536852 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000  0.0000 0.0000   
536974 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000  0.0000 0.0000   
537065 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000  0.0000 0.0000   
537463 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 10.0000 0.0000   

商品番号    20724   20725   20726  20727   20728  20749  20750  20751  20767  \
発注番号                                                                       
536370 0.0000  0.0000  0.0000 0.0000  0.0000 0.0000 0.0000 0.0000 0.0000   
536852 0.0000  0.0000  0.0000 0.0000  0.0000 0.0000 0.0000 0.0000 0.0000   
536974 0.0000 10.0000 10.0000 0.0000  0.0000 2.0000 2.0000 0.0000 0.0000   
537065 0.0000 10.0000 20.0000 0.0000 10.0000 2.0000 6.0000 0.0000 0.0000   
537463 0.0000  0.0000  0.0000 0.0000  0.0000 0.0000 0.0000 0.0000 0.0000   

商品番号    20777  20819  20832  20866  20914  20961  20963  20966  20967  20971  \
発注番号                                                                           
536370 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000   
536852 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000   
536974 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000   
537065 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000   
537463 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000   

商品番号    20972  20973  20974  20975  20977  20978  20979  20981  20983  20984  \
発注番号                                                                           
536370 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000   
536852 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000   
536974 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000   
537065 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000   
537463 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000   

商品番号    20992  20996  21003  21012  21026  21027  21030  21034   21035  21039  \

In [18]:
# 集計結果が正か0かでTrue/Falseを設定
basket_df = w2.apply(lambda x: x > 0)

# 結果確認
display(basket_df.head())

商品番号    10002  10120  10125  10135  11001  15036  15039  16012  16048  16218  \
発注番号                                                                           
536370   True  False  False  False  False  False  False  False  False  False   
536852  False  False  False  False  False  False  False  False  False  False   
536974  False  False  False  False  False  False  False  False  False  False   
537065  False  False  False  False  False  False  False  False  False  False   
537463  False  False  False  False  False  False  False  False  False  False   

商品番号    16219  16225  16236  16237  16238  17174  20615  20617  20658  20665  \
発注番号                                                                           
536370  False  False  False  False  False  False  False  False  False  False   
536852  False  False  False  False  False  False  False  False  False  False   
536974  False  False  False  False  False  False  False  False  False  False   
537065  False  False  False  False   True  False  False  False  False   True   
537463  False  False  False  False  False  False  False  False  False  False   

商品番号    20668  20674  20675  20676  20677  20679  20681  20682  20684  20685  \
発注番号                                                                           
536370  False  False  False  False  False  False  False  False  False  False   
536852  False  False  False  False  False  False  False  False  False  False   
536974  False  False  False  False  False   True  False  False  False  False   
537065  False  False  False  False  False  False  False  False  False  False   
537463  False  False  False  False  False  False   True   True  False  False   

商品番号    20686  20702  20704  20711  20712  20713  20717  20718  20719  20723  \
発注番号                                                                           
536370  False  False  False  False  False  False  False  False  False  False   
536852  False  False  False  False  False  False  False  False  False  False   
536974  False  False  False  False  False  False  False  False  False  False   
537065  False  False  False  False  False  False  False  False  False  False   
537463  False  False  False  False  False  False  False  False   True  False   

商品番号    20724  20725  20726  20727  20728  20749  20750  20751  20767  20777  \
発注番号                                                                           
536370  False  False  False  False  False  False  False  False  False  False   
536852  False  False  False  False  False  False  False  False  False  False   
536974  False   True   True  False  False   True   True  False  False  False   
537065  False   True   True  False   True   True   True  False  False  False   
537463  False  False  False  False  False  False  False  False  False  False   

商品番号    20819  20832  20866  20914  20961  20963  20966  20967  20971  20972  \
発注番号                                                                           
536370  False  False  False  False  False  False  False  False  False  False   
536852  False  False  False  False  False  False  False  False  False  False   
536974  False  False  False  False  False  False  False  False  False  False   
537065  False  False  False  False  False  False  False  False  False  False   
537463  False  False  False  False  False  False  False  False  False  False   

商品番号    20973  20974  20975  20977  20978  20979  20981  20983  20984  20992  \
発注番号                                                                           
536370  False  False  False  False  False  False  False  False  False  False   
536852  False  False  False  False  False  False  False  False  False  False   
536974  False  False  False  False  False  False  False  False  False  False   
537065  False  False  False  False  False  False  False  False  False  False   
537463  False  False  False  False  False  False  False  False  False  False   

商品番号    20996  21003  21012  21026  21027  21030  21034  21035  21039  21

## コラム 商品名辞書の作成

In [19]:
# 「商品番号」と「商品説明」のみを抽出
w3 = df2[['商品番号', '商品説明']].drop_duplicates()

# 商品番号と商品名をすべて文字列に変換
w3['商品番号'] = w3['商品番号'].astype('str')
w3['商品説明'] = w3['商品説明'].astype('str')

# 商品番号をインデックスに
w3 = w3.set_index('商品番号')
display(w3.head())
# 冒頭の5つの商品コード抽出
item_list1 = w3.index[:5]

# w3に対する検索結果
display(w3.loc[item_list1])
# 大文字だけのテキストの抽出

# 作業用にコピー
w4 = w3.copy()

# すべて大文字名称の行のみ残す
w4 = w4[w4['商品説明'].map(lambda x: x.isupper())]

# 先ほど作ったitem_list1で結果確認
display(w4.loc[item_list1])

# 一番長い名称の抽出

# 作業用にコピー
w5 = w4.copy()

# 「文字数」列の追加
w5['文字数'] = w5['商品説明'].map(len)

# ソート
w5 = w5.sort_values(['商品番号', '文字数'], ascending=[True, False])

# 商品番号ごとに一番長い名称を抽出してitem_dictに代入
item_dict = w5.groupby('商品番号')['商品説明'].agg(lambda x: x[0])

# 先ほど作ったitem_list1で結果確認
display(item_dict.loc[item_list1])

商品説明
商品番号                                       
85123A   WHITE HANGING HEART T-LIGHT HOLDER
71053                   WHITE METAL LANTERN
84406B       CREAM CUPID HEARTS COAT HANGER
84029G  KNITTED UNION FLAG HOT WATER BOTTLE
84029E       RED WOOLLY HOTTIE WHITE HEART.

商品説明
商品番号                                       
85123A   WHITE HANGING HEART T-LIGHT HOLDER
85123A                                    ?
85123A          wrongly marked carton 22804
85123A   CREAM HANGING HEART T-LIGHT HOLDER
71053                   WHITE METAL LANTERN
71053          WHITE MOROCCAN METAL LANTERN
84406B       CREAM CUPID HEARTS COAT HANGER
84406B        incorrectly made-thrown away.
84406B                                    ?
84406B                                  nan
84029G  KNITTED UNION FLAG HOT WATER BOTTLE
84029G                                  nan
84029E       RED WOOLLY HOTTIE WHITE HEART.
84029E                                  nan

商品説明
商品番号                                       
85123A   WHITE HANGING HEART T-LIGHT HOLDER
85123A   CREAM HANGING HEART T-LIGHT HOLDER
71053                   WHITE METAL LANTERN
71053          WHITE MOROCCAN METAL LANTERN
84406B       CREAM CUPID HEARTS COAT HANGER
84029G  KNITTED UNION FLAG HOT WATER BOTTLE
84029E       RED WOOLLY HOTTIE WHITE HEART.

商品番号
85123A     WHITE HANGING HEART T-LIGHT HOLDER
71053            WHITE MOROCCAN METAL LANTERN
84406B         CREAM CUPID HEARTS COAT HANGER
84029G    KNITTED UNION FLAG HOT WATER BOTTLE
84029E         RED WOOLLY HOTTIE WHITE HEART.
Name: 商品説明, dtype: object

## 5.4.6 アルゴリズムの選択と分析

In [22]:
# ライブラリの読み込み
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [26]:
# アプリオリによる分析
freq_items1 = apriori(basket_df, min_support=0.06, use_colnames=True)

#　結果確認
display(freq_items1.sort_values('support', ascending=False).head())

# itemset数確認
print(freq_items1.shape[0])

support itemsets
61   0.7653   (POST)
52   0.1888  (23084)
14   0.1811  (21731)
37   0.1709  (22554)
39   0.1684  (22556)

134


In [27]:
# アソシエーションルールの抽出
a_rules1 = association_rules(freq_items1, metric='lift', min_threshold=1)

# リフト値でソート
a_rules1 = a_rules1.sort_values('lift', ascending=False).reset_index(drop=True)

# 結果確認
display(a_rules1.head(10))

# ルール数確認
print(a_rules1.shape[0])

antecedents     consequents  antecedent support  consequent support  \
0         (23256)         (23254)              0.0689              0.0714   
1         (23254)         (23256)              0.0714              0.0689   
2         (22727)  (22728, 22726)              0.0944              0.0740   
3  (22728, 22726)         (22727)              0.0740              0.0944   
4   (22726, POST)         (22727)              0.0842              0.0944   
5         (22727)   (22726, POST)              0.0944              0.0842   
6         (22726)  (22728, 22727)              0.0969              0.0740   
7  (22728, 22727)         (22726)              0.0740              0.0969   
8         (22726)         (22727)              0.0969              0.0944   
9         (22727)         (22726)              0.0944              0.0969   

   support  confidence    lift  leverage  conviction  
0   0.0638      0.9259 12.9630    0.0589     12.5357  
1   0.0638      0.8929 12.9630    0.0589      8.6905  
2   0.0638      0.6757  9.1333    0.0568      2.8552  
3   0.0638      0.8621  9.1333    0.0568      6.5657  
4   0.0714      0.8485  8.9894    0.0635      5.9770  
5   0.0714      0.7568  8.9894    0.0635      3.7650  
6   0.0638      0.6579  8.8929    0.0566      2.7068  
7   0.0638      0.8621  8.8929    0.0566      6.5472  
8   0.0791      0.8158  8.6430    0.0699      4.9162  
9   0.0791      0.8378  8.6430    0.0699      5.5689

206


In [28]:
# 関連性の高い商品の商品コードをリストアップ
item_list = ['23254', '23256', '22726', '22727', '22728']

# 商品名を確認
for item in item_list:
    print(item, item_dict[item])

23254 CHILDRENS CUTLERY DOLLY GIRL 
23256 CHILDRENS CUTLERY SPACEBOY 
22726 ALARM CLOCK BAKELIKE GREEN
22727 ALARM CLOCK BAKELIKE RED 
22728 ALARM CLOCK BAKELIKE PINK


## 5.4.7 チューニング

In [30]:
# アプリオリによる分析
freq_items2 = apriori(basket_df, min_support=0.065, use_colnames=True)

# アソシエーションルールの抽出
a_rules2 = association_rules(freq_items2, metric='lift', min_threshold=1)

# リフト値でソート
a_rules2 = a_rules2.sort_values('lift', ascending=False).reset_index(drop=True)

# 結果確認
display(a_rules2.head(10))

antecedents    consequents  antecedent support  consequent support  \
0        (22727)  (22726, POST)              0.0944              0.0842   
1  (22726, POST)        (22727)              0.0842              0.0944   
2        (22727)        (22726)              0.0944              0.0969   
3        (22726)        (22727)              0.0969              0.0944   
4        (22726)  (22727, POST)              0.0969              0.0867   
5  (22727, POST)        (22726)              0.0867              0.0969   
6  (22728, POST)        (22727)              0.0893              0.0944   
7        (22727)  (22728, POST)              0.0944              0.0893   
8        (22727)        (22728)              0.0944              0.1020   
9        (22728)        (22727)              0.1020              0.0944   

   support  confidence   lift  leverage  conviction  
0   0.0714      0.7568 8.9894    0.0635      3.7650  
1   0.0714      0.8485 8.9894    0.0635      5.9770  
2   0.0791      0.8378 8.6430    0.0699      5.5689  
3   0.0791      0.8158 8.6430    0.0699      4.9162  
4   0.0714      0.7368 8.4954    0.0630      3.4704  
5   0.0714      0.8235 8.4954    0.0630      5.1173  
6   0.0663      0.7429 7.8703    0.0579      3.5218  
7   0.0663      0.7027 7.8703    0.0579      3.0633  
8   0.0740      0.7838 7.6811    0.0643      4.1531  
9   0.0740      0.7250 7.6811    0.0643      3.2931

In [31]:
# 調査対象の集合
t_set = set([23254, 23256])

# 1回目の分析freq_items1から該当行を抽出
idx1 = freq_items1['itemsets'].map(lambda x: not x.isdisjoint(t_set))
item1 = freq_items1[idx1]

# 2回目の分析freq_items2から該当行を抽出
idx2 = freq_items2['itemsets'].map(lambda x: not x.isdisjoint(t_set))
item2 = freq_items2[idx2]

# 結果確認
display(item1)
display(item2)

support        itemsets
58    0.0714         (23254)
59    0.0689         (23256)
118   0.0638  (23256, 23254)

support itemsets
53   0.0714  (23254)
54   0.0689  (23256)